<a href="https://www.kaggle.com/code/ravi20076/playgrounds4e08-dataorganization?scriptVersionId=190679775" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%%capture

import pandas as pd

!pip install polars==1.2.1
import polars as pl
import polars.selectors as cs
import numpy as np
from pprint import pprint

from gc import collect

# **FOREWORD**

This kernel organizes the train and original data and presents a smaller file to use in subsequent experiments <br>
It stores the files in parquet for easy usage subsequently 

# **ORIGINAL DATA**

In [2]:
%%capture

!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# data (as pandas dataframes) 
X = mushroom.data.features 
y = mushroom.data.targets 

original1 = pd.concat([pd.DataFrame(X), pd.DataFrame(y)], axis=1)
original1.index.name = "id"


# **SUPPLEMENTARY ORIGINAL DATA**

In [3]:
%%time 

# Source- https://archive.ics.uci.edu/dataset/848/secondary+mushroom+dataset
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
secondary_mushroom = fetch_ucirepo(id=848) 
  
# data (as pandas dataframes) 
X = secondary_mushroom.data.features 
y = secondary_mushroom.data.targets 
  
original = pd.concat([pd.DataFrame(X), pd.DataFrame(y)], axis=1)
original.index.name = "id"

CPU times: user 271 ms, sys: 65.4 ms, total: 336 ms
Wall time: 2.68 s


# **COMPETITION DATA**

In [4]:
%%time 

train  = pd.read_csv(f"/kaggle/input/playground-series-s4e8/train.csv").set_index("id")
test   = pd.read_csv(f"/kaggle/input/playground-series-s4e8/test.csv").set_index("id")
sub_fl = pd.read_csv(f"/kaggle/input/playground-series-s4e8/sample_submission.csv")

print(f"\n---> Shapes = {train.shape} | {test.shape} | {original.shape}\n\n")

with np.printoptions(linewidth = 100):
    print(f"---> Train set columns")
    pprint(np.array(train.columns))
    
    print(f"\n\n---> Original set columns")
    pprint(np.array(original.columns))   
    
    print(f"\n\n---> Mismatched column names")
    pprint(set(original.columns).difference(set(train.columns)))
    
original = original.rename(columns = {'poisonous' : 'class'})

# Adjusting the column names across datasets
train.columns    = train.columns.str.replace("-","")
test.columns     = test.columns.str.replace("-","")
original.columns = original.columns.str.replace("-","")

with np.printoptions(linewidth = 100):
    print(f"\n\n---> Train set columns renamed")
    pprint(np.array(train.columns))
    
    print(f"\n\n---> Original set columns renamed")
    pprint(np.array(original.columns)) 
    
    print(f"\n\n---> Mismatched column names")
    pprint(set(original.columns).difference(set(train.columns)))
    
    print(f"\n\n---> Common column names")
    pprint(set(original.columns).intersection(set(train.columns)))    

print();
collect();


---> Shapes = (3116945, 21) | (2077964, 20) | (61069, 21)


---> Train set columns
array(['class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
       'gill-attachment', 'gill-spacing', 'gill-color', 'stem-height', 'stem-width', 'stem-root',
       'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type',
       'spore-print-color', 'habitat', 'season'], dtype=object)


---> Original set columns
array(['cap-diameter', 'cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
       'gill-attachment', 'gill-spacing', 'gill-color', 'stem-height', 'stem-width', 'stem-root',
       'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type',
       'spore-print-color', 'habitat', 'season', 'class'], dtype=object)


---> Mismatched column names
set()


---> Train set columns renamed
array(['class', 'capdiameter', 'capshape', 'capsurface', 'capcolor', 'doesbruiseorbleed',
       'gillattachment', 'gillsp

In [5]:
%%time 

train.to_parquet("train.parquet")
test.to_parquet("test.parquet")
sub_fl.set_index("id").to_parquet("sample_submission.parquet")
original.to_parquet("original.parquet")

%reset -f

CPU times: user 12.4 s, sys: 801 ms, total: 13.2 s
Wall time: 13.1 s


# **CHECKS**

In [6]:
import pandas as pd
display(pd.read_parquet("sample_submission.parquet").head(10))

,class
id,
3116945,e
3116946,e
3116947,e
3116948,e
3116949,e
3116950,e
3116951,e
3116952,e
3116953,e
